# **Welcome to Bittensor!**

(This notebook is supposed to run with pip-installed version of bittensor)

Bittensor is a decentralized market that enables individuals to monetize intelligence production from any computer anywhere in the world. Intelligence production is validated by the other peers in the network and rewarded through token inflation. Consumers stake this currency to gain access to the produced knowledge. Bittensor is collectively-run, open-source, and open-access.

For more info...

* [Visit our website](https://www.bittensor.com/)
* [Read our paper](https://arxiv.org/abs/2003.03917)
* [Check out the code](https://github.com/opentensor/BitTensor)
* [Talk to us!](https://discord.gg/3rUr6EcvbB)

### **Some basic terminologies**
Before moving forward, know these terms!

1. Tau: The currency name.
2. Neuron/node/peers/endpoint: Server that mines Tau, each server owns a machine learning model, which we call nucleus.
3. Network/chain: The whole group of people that miners Tau.
4. Weight: The scoring that other neurons in the network gives you. 
5. Stake: The higher the stake, the more Tau you own. It is also determined by your weight and the time you stay in the network.

### **Structure of this tutorial**
1. [Quick start!](#quickStart) 
3. [Wallet](#wallet)
2. [Metagraph](#metagraph)
6. [Subtensor](#subtensor)
4. [Dendrite](#dendrite)
5. [Axon](#axon)



### **Let's get the bittensor package ready!**

In [ ]:
# ---- install the package ----
! pip install bittensor

In [ ]:
# ---- import ----
import bittensor
import torch

# ---- Enable logging from bittensor ----
bittensor.logging(debug = False)

<a id='quickStart'></a>
# **Quick start!** 

#### **Client side**

On the client side, we can send request for a representation of our data from each of the chosen node.

In [ ]:
# ---- set up a wellet ----
wallet = bittensor.wallet().create()

# ---- sync to our metagraph (this will be further explained) ----
graph = bittensor.metagraph().sync()

# ---- querying for representation by calling dendrite ----
representations, _ = bittensor.dendrite( wallet = wallet ).forward_text (
    endpoints = graph.endpoints,
    inputs = "The quick brown fox jumped over the lazy dog"
)
# ---- checking out the representations ----
# representations = Tensor with shape (number of nodes online, 9, 512)

### Server side

On the server side, we get to mine Tau using our own model.

In [ ]:
from transformers import BertModel, BertConfig

# ---- set up a wallet ----
wallet = bittensor.wallet().create()

# ---- set up the forward function ----
model = BertModel(BertConfig())

def forward ( pubkey, inputs_x, modality ):
    return model( inputs_x ).narrow(2, 0, bittensor.__network_dim__)

# ---- subscribe to axon, that handles forward and backward request from other neurons ----
axon = bittensor.axon (
    wallet = wallet,
    forward_text = forward,
).start().subscribe()

<a id='wallet'></a>
# **Wallet**

A wallet has the following properties.
* name
* coldkey: Used to store, transfer, and stake tokens. It is "cold" because it is not loaded into the miner and remains encrypted on the device. 
* hotkey: Used by the miner to subscribe and set weights. It is "hot" because it is loaded into the running software (which can be insecure). It does not have permission to move funds.

NOTE: Remember to save the mnemonic of the cold key and hot key for regenerating your password or hot key.

In [ ]:
# ---- initialize a wallet object ----
# if you want to creat a new wallet, choose a new wallet name, else just choose an existing wallet name 

wallet = bittensor.wallet(name = 'wallet-name')
wallet.create()

<a id='metagraph'></a>
# **Metagraph** 
Metagraph is an object that maintains the chain state as a torch.nn.Module. First, setp up the metagraph object.

In [ ]:
# ---- Initializing the metagraph object ----
meta = bittensor.metagraph()

# ---- Synchronise states in meta graph with the akatsuki network, aka the chain----
meta = meta.sync()

print("---- Metagraph keys ----\n\n", meta.state_dict().keys())

### **Block info**
* n: the number of neurons in the network
* block: the block id 
* tau (T): the token inflation rate per block

In [ ]:
print("---- Block info ----\n")
print("n:\t\t", meta.n.item())
print("block ID:\t", meta.block.item())
print("tau:\t\t", meta.tau[0])

### **Neurons' info**
* uids: the user id of each neurons
* last_update: the last time the the users emit to the chain
* endpoints: the encoded ip addresses, hotkey, coldkey of other neurons. Can be retrieved by bittensor.endpoint.from_tensor().

In [ ]:
print("---- Neurons' info (of the first 5 neurons) ----\n")
print("uids :\t", meta.uids[0:5])
print("last_update :\t", meta.last_update[0:5])
print("\nendpoint (of the first neuron):\n\n", meta.endpoints[0])
print("\nip address/hotkey/coldkey (of the first neuron): \n\n", bittensor.endpoint.from_tensor(meta.endpoints[0]))

print("\nhotkeys: \n\n", meta.hotkeys[0:5])
print("\ncoldkeys: \n\n", meta.coldkeys[0:5])
print("\naddresses: \n\n", meta.addresses[0:5])

### **Neuron's scorings**
* weight (W): the weight of of the network, which is an n*n matrix. W_{i,j} indicates the locally calculated score from uid[i] to uid[j]. 
* rank (R): R = W<sup>T</sup> &middot; S. It also controls the priority of message to be read by peers.
* trust (T): T = $\sigma$ (C <sup>T</sup> S), where C is the connectivity matrix of size n*n. C<sub>i,j</sub> indicates the number of expectation in time that peer i reach peer j.
* incentive (I): I = W<sup>T</sup> S &middot; $\sigma$ (C <sup>T</sup> S)
* stake (S): the stake of each neuron, updated with function S<sub>t+1</sub> = S<sub>t</sub> + (&tau; R ||S||) / ||R|| 

In [ ]:
print("---- Neurons' scoring (of and within the first 5 neurons) ----\n")
print("Weight:\n", meta.W[0:5, 0:5])
print("Stake:\t", meta.S[0:5])
print("Rank:\t", meta.R[0:5])
print("Incentive:\t", meta.I[0:5])

### **Saving and loading the metagraph** 

In [ ]:
meta.save()
meta.load()

meta.save_to_path('~/.bittensor/', 'network-name.pt')
meta.load_from_path('~/.bittensor/network-name.pt')

### **Use Cases**

In [ ]:
# TODO: maybe list more use cases

# ---- getting the endpoints with stake at the top 90% ----
meta.endpoints[meta.S > torch.quantile(meta.S, 0.9)]

# ---- querying the stake of a neuron from its pubkey(hotkey) ----
# stake_of_caller = meta.S[meta.hotkeys.index(pubkey)]

<a id='subtensor'></a>
# **Subtensor**

It handles the interactions with the subtensor chain.

In [ ]:
# ---- initialize ----
subtensor = bittensor.subtensor()

# ---- Subscribes an bittensor endpoint to the substensor chain. ----
subtensor.subscribe(wallet, axon.external_ip, axon.external_port, 0)

# ---- shall we include these functions as well? ----

# subtensor.connect()
# subtensor.set_weights()
# subtensor.get_balances()

<a id='dendrite'></a>
# **Dendrite**

The dendrite object is for questing representation of your data from other neurons.

In [ ]:
# the projected size of representation
bittensor.__network_dim__

In [ ]:
# ---- initializing a dendrite object ----
# note: it is not required to attach a wallet to the dendrite object 
dend = bittensor.dendrite(wallet = wallet)

In [ ]:
# ---- specify the endpoints that you want to send request to ----
# here we select the top 10% endpoints that has the most stake 
endpoints = meta.endpoints[meta.S > torch.quantile(meta.S, 0.9)]

# ---- retrieving the represenataions from other 
representation = dend.forward_text(endpoints, "The quick brown fox jumped over the lazy dog" )

In [ ]:
# ---- checking on the representation ----
representation[0]

<a id='axon'></a>
# **Axon**

An axon object that handles forward and backward request from other neurons.

In [ ]:
# ---- first, define a model and a forward function to be passed to the Axon object ----
# the resulted model representation has to be narrowed to our network_dim 

from transformers import BertModel, BertConfig
model = BertModel(BertConfig())

def forward ( pubkey, inputs_x, modality ):
    return model( inputs ).narrow(2, 0, bittensor.__network_dim__)

In [ ]:
# ---- initialize the axon object ----
# ---- start the axon object, but other neurons still can't find you!!! ----
# ---- subscribe the axon object to the network. Okay! other neurons can see you now! ---- 
axon_ = bittensor.axon (
    wallet = wallet,
    forward_text = forward,
).start().subscribe()

In [ ]:
# Bug?
axon_ = bittensor.axon (
    wallet = wallet,
    forward_text = forward,
)
axon.start()
axon.subscribe()

In [ ]:
# ---- check your subscribed ip and port ----
print(f"{axon.external_ip}:{axon.external_port}" )